# Thermal pressurization in an undrained porous solid

*[Written by: Maxime Pierre, 2024]* \
\
Let us come back to our first example and complicate it a bit: we will consider an undrained, unconfined thermoporoelastic solid subject to a heat source. \
Density of water will be taken as a linear function of temperature and pressure (from IAWPS data): 
$$
\rho_f = 1000 + 0.487p - 0.0284(T-T_0).
$$
The usual thermoporoelastic equations form read:
$$
\begin{array}{}
\sigma = K\varepsilon - bp - 3\alpha_s KT, \\
\phi-\phi_0 = b\varepsilon + \frac{p}{N} - 3\alpha_s(b-\phi_0)T, \\
S - S_0 = 3\alpha_s K \varepsilon - 3\alpha_s(b-\phi_0)p + C_s\frac{T}{T_0}.
\end{array}
$$

Boundary conditions amount to the following:

$$
\begin{array}{}
\sigma=0,\\
m_f = m_f(0),\\
S-S_0 = \frac{Q}{T_0}
\end{array}
$$




In [76]:
import numpy as np
import matplotlib.pyplot as plt

# Material constants
K = 1e10
b = 0.4
N = 1e12
phi_0 = 0.3
alpha_s = 0
C_s = 2e6

# Loading
Q = 2e9

T0 = 293.15

drho_f_dp = 0.487
drho_f_dT = -0.0284


def rho_f(p, T):
    return 1000 + drho_f_dp * p + drho_f_dT * T


def F(X):
    eps, p, T = X
    return np.array(
        [
            K * eps - b * p - 3 * alpha_s * K * T,
            rho_f(p, T) * (phi_0 + b * eps + p / N - 3 * alpha_s * (b - phi_0) * T)
            - rho_f(0, 0) * phi_0,
            3 * alpha_s * K * eps
            - 3 * alpha_s * (b - phi_0) * p
            + C_s * T / T0
            - Q / T0,
        ]
    )


def J(X):
    eps, p, T = X

    dsig_deps = K
    dsig_dp = -b
    dsig_dT = -3 * alpha_s * K
    dmf_deps = rho_f(p, T) * b
    dmf_dp = (
        drho_f_dp * (phi_0 + b * eps + p / N - 3 * alpha_s * (b - phi_0) * T)
        + rho_f(p, T) * 1 / N
    )
    dmf_dT = drho_f_dT * (
        phi_0 + b * eps + p / N - 3 * alpha_s * (b - phi_0) * T
    ) + rho_f(p, T) * (-3 * alpha_s * (b - phi_0))
    dS_deps = 3 * alpha_s * K
    dS_dp = -3 * alpha_s * (b - phi_0)
    dS_dT = C_s / T0
    return np.array(
        [
            [dsig_deps, dsig_dp, dsig_dT],
            [dmf_deps, dmf_dp, dmf_dT],
            [dS_deps, dS_dp, dS_dT],
        ]
    )


def newton_method(residu, jacobian, initial_guess, tolerance, max_iteration=50):
    X_n = initial_guess
    X = [initial_guess]
    iteration = 0

    while np.linalg.norm(residu(X_n)) > tolerance and iteration < max_iteration:
        X_n -= np.dot(np.linalg.inv(jacobian(X_n)), residu(X_n))

        X.append(X_n.copy())
        iteration += 1
        print(
            "Iteration {}: X_{} = {}, residual norm = {}".format(
                iteration, iteration, X_n, np.linalg.norm(residu(X_n))
            )
        )

    if np.linalg.norm(residu(X_n)) > tolerance:  # Not converged
        print("Did not converge after {} iterations.".format(iteration))
    else:  # Converged
        print("Converged in {} iterations.".format(iteration))

    return np.array(X)


X_0 = [0, 0, 0]
eps = 1e-15

result = newton_method(F, J, X_0, eps)

Iteration 1: X_1 = [2.3326486e-09 5.8316215e+01 1.0000000e+03], residual norm = 5.695433295429594e-14
Iteration 2: X_2 = [2.3326486e-09 5.8316215e+01 1.0000000e+03], residual norm = 0.0
Converged in 2 iterations.


In [77]:
eps, p, T = result[-1]